# Voluntary Problem Set

This problem set allows you to play around with concepts from class and to solve some smaller problems on your own.

# Topic: 2nd Moment and First Moment Shocks to Equity returns

you will learn about

- Measuring Uncertainty in Equity Markets
- Gram-Schmidt Orthogonalization
- AR(1) regressions
- principal component analysis
- fitting stochastic volatility models
- extracting volatility shocks
- testing the so called 'spanning hypothesis' for equity volatility

# Tasks

We want to learn about the similarity and dissimilarity of US and EU uncertainty. Moreover, we want to learn whether return shocks span volatility shocks.

**Hint:** When working with principal components, use normalized data (i.e. 'z')
$$
\\
$$

**A. Load-in US and EU data**. 

- ignore the equal weight portfolio (i.e. focus on individual stocks only) 

- Ensure that start and end date are aligned across both data sets 
Hint: (i.e. Full Outer Joint) 
https://www.datacamp.com/community/tutorials/joining-dataframes-pandas

 
$$
\\
$$

**B. Extract the first two principal components of the international return panel. Ensure that these are displayed with a mean of zero and a unit volatility**

- How important are the first and the second principal components?

- Plot the time series of both principal components


$$
\\
$$

**C. Compute PC1 of the EU returns and PC1 of the US returns. Ensure that these are displayed with a mean of zero and a unit volatility.**

- Plot both PCs

-  How large is the unconditional correlation of the EU and the US first principal component of returns?

- Regress PC1 of the international return panel onto PC1 of EU equity. How large ist the adj-R2? How large are the point estimates and t-stats?

- Regress PC1 of the international return panel onto the Gram-Schmidt orthogonalized first principal components of US and EU equity. Treat the common information in PC1(US equity) and PC1(EU equity) as US only information. How much additional variance explained does EU information (that is orthogonal to US information) add for explaining variations in PC1 of the international return panel?

Hint on Gram-Schmidt: See example 6:35min/8:11min in https://www.youtube.com/watch?v=4Rzn-Pfvof4&list=PLyQSjcv8LwAHcUWCG-zRWbzuczxa0hB3n&index=3

- Regress PC1 of the international return panel onto PC1 of US equity. How large ist the adj-R2? How large are the point estimates and t-stats?
    
$$
\\
$$

**D.  Filter time-series of variance** for
\begin{itemize}
\item PC1(international equity)  
\item PC1(US equity)
\item PC1(EU equity)
\end{itemize}
using univariate stochastic volatility models.

- What are the pariwise correlations of the respective variances? Interpret results.

- Plot the filtered variances in one graph. Looking at uncertainties, which currency area was the epicenter of the Covid pandemic?
$$
\\
$$

**E. Extract the time-series of volatility shocks** for 
\begin{itemize}
\item PC1(international equity)  
\item PC1(US equity)
\item PC1(EU equity)
\end{itemize}

Hint: look carefully at the parametrization of a SVM.

- By how much do US volatility shocks outweigh ES volatility shocks?  

- What are the pairwise correlations of volatility shocks?
$$
\\
$$

**F. Extract the time-series of return shocks** for 
\begin{itemize}
\item PC1(international equity)  
\item PC1(US equity)
\item PC1(EU equity)
\end{itemize}

What are the pariwise correlations of the return shocks? Interpret results.
 
Hint: Assume the mean equation for these quantities follows an AR(1). 
$$
\\
$$

**G. Do return shocks span volatility shocks? To answer the question, regress volatility shocks** of
\begin{itemize}
\item PC1(international equity)  
\item PC1(US equity)
\item PC1(EU equity)
\end{itemize}

onto their respective return shocks. How large are the respective R2, point (t-stat) estimates? Interpret results.